# CS 20 : TensorFlow for Deep Learning Research
## Lecture 04 : Eager execution
### Custon training walkthrough
Categorizing Iris flowers by species by using Tensorflow's eager execution.

This guide uses these high-level TensorFlow concepts:

* Enable an [eager execution](https://www.tensorflow.org/guide/eager?hl=ko) development environment,
* Import data with the [Datasets API](https://www.tensorflow.org/guide/datasets?hl=ko)
* Build models and layers with TensorFlow's [Keras API](https://keras.io/getting-started/sequential-model-guide/)  
  
  
* Reference
    + https://www.tensorflow.org/tutorials/eager/custom_training_walkthrough?hl=ko

### Setup

In [1]:
from __future__ import absolute_import, division, print_function
import os, sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

tf.enable_eager_execution()

print(tf.__version__)

1.12.0


### Create a model

In [2]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation = tf.nn.relu, input_shape = (4,)))
model.add(tf.keras.layers.Dense(10, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3))

In [3]:
def loss_fn(model, features, label):
    score = model(features)
    return tf.losses.sparse_softmax_cross_entropy(labels = label, logits = score)

In [4]:
print(model.trainable_variables) # different from tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'dense/kernel:0' shape=(4, 10) dtype=float32, numpy=
array([[-0.6016106 ,  0.21193182,  0.15714651,  0.15312278, -0.28133038,
         0.00746506, -0.148848  , -0.1853224 ,  0.18714899, -0.25901568],
       [ 0.02185094, -0.08642298,  0.5632899 , -0.43347245,  0.1226458 ,
        -0.49662638, -0.23893097,  0.49837744,  0.39349937, -0.3385544 ],
       [-0.27739072, -0.31779853,  0.64550364,  0.33254373,  0.23269486,
        -0.2784947 ,  0.06808609,  0.21770716,  0.23193008, -0.16453654],
       [-0.5709686 , -0.24807453,  0.17099643, -0.20884967, -0.2562401 ,
         0.34987652,  0.52932966, -0.1443029 ,  0.42169094,  0.46493506]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>, <tf.Variable 'dense_1/kernel:0' shape=(10, 10) dtype=float32, numpy=
array([[ 0.4289506 ,  0.10450268, -0.04721606, -0.43415242, -0.47025183,
        -0.361524  ,  0.0418207 , -0.0864554 , -0.055

### Import and parse the training dataset

In [5]:
os.listdir('../data/lecture04/')

['iris_test.csv', 'iris_training.csv']

In [6]:
# define parsing function
def parse_single_example(record):
    decoded = tf.decode_csv(record, [[.0],[.0],[.0],[.0],[]])
    features = decoded[:4]
    label = tf.cast(x = decoded[4], dtype = tf.int32)
    return features, label

In [7]:
epochs = 10
batch_size = 8
learning_rate = .03
tr_dataset = tf.data.TextLineDataset(filenames = '../data/lecture04/iris_training.csv')
tr_dataset = tr_dataset.map(parse_single_example)
tr_dataset = tr_dataset.shuffle(200).batch(batch_size = batch_size)
opt = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
global_step = tf.Variable(0.)

### Train the model

In [8]:
for epoch in range(epochs):
    avg_loss = 0
    tr_step = 0
    
    for mb_x, mb_y in tr_dataset:
        with tf.GradientTape() as tape:
            tr_loss = loss_fn(model, mb_x, mb_y)
        grads = tape.gradient(tr_loss, model.variables)
        opt.apply_gradients(zip(grads, model.variables), global_step = global_step)
        
        avg_loss += tr_loss
        tr_step += 1
    else:
        avg_loss /= tr_step
    
    print('epoch : {:3}, ce_loss : {:.3f}'.format(epoch + 1, avg_loss))

epoch :   1, ce_loss : 1.154
epoch :   2, ce_loss : 0.829
epoch :   3, ce_loss : 0.706
epoch :   4, ce_loss : 0.591
epoch :   5, ce_loss : 0.553
epoch :   6, ce_loss : 0.553
epoch :   7, ce_loss : 0.449
epoch :   8, ce_loss : 0.451
epoch :   9, ce_loss : 0.514
epoch :  10, ce_loss : 0.395


### Evaluate the model on the test dataset

In [9]:
tst_dataset = tf.data.TextLineDataset(filenames = '../data/lecture04/iris_test.csv')
tst_dataset = tst_dataset.map(parse_single_example)
tst_dataset = tst_dataset.batch(batch_size = 30)

In [10]:
tst_x, tst_y = next(iter(tst_dataset))
tst_yhat = tf.argmax(model(tst_x), axis = -1, output_type = tf.int32) 

In [11]:
print('test accuracy : {:.2%}'.format(np.mean(tf.equal(tst_y, tst_yhat))))

test accuracy : 96.67%
